In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import time
import os
import pandas as pd

d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()
  
def count_pos(arr):
  count = 0
  for num in arr:
    if num > 0:
        count += 1
  print("Number of positive elements in the array:", count)

In [6]:
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    denominator = dx_dt**2 + dy_dt**2
    curvature = np.zeros_like(denominator)
    nonzero_denominator = denominator != 0
    curvature[nonzero_denominator] = (dx_dt[nonzero_denominator] * d2y_dt2[nonzero_denominator] - d2x_dt2[nonzero_denominator] * dy_dt[nonzero_denominator]) / denominator[nonzero_denominator]**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i, x, y, r, memo):
    # Check if the result has already been computed
    if (i, r) in memo:
        return memo[(i, r)]
    # Compute the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    # Memoize the result
    memo[(i, r)] = int_curv
    return int_curv


def get_curvatures(contours, radius_arr):
    curvature_arr = []
    # Loop over each contour
    for contour in contours:
        curr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the area measure for each point in the contour
        memo = {}
        for radius in radius_arr:
            curr_int = []
            for i in range(len(contour)):
                curr_int.append(get_integral_curvature(i, x, y, radius, memo))
            curr.append(curr_int)
        curvature_arr.append(curr)

    return curvature_arr

    
def to_hist(currvature_arr):
  currvature_arr = np.array(currvature_arr)
  x = np.arange(currvature_arr.shape[1]) # x-axis is the number of contour points
  y = np.arange(currvature_arr.shape[0]) # y-axis is the different scales
  X, Y = np.meshgrid(x, y)
  hist_values = []


In [7]:
radius_arr = np.arange(10, 131, 5)

In [9]:
# specify path to the folder containing image files
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/lab_/4"
# path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"

counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue

    if counter == 1:
        break
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        try:
            
            start_time = time.time()
            # Load the image
            img = cv2.imread(filepath)
            # set the new size
            new_size = (250, 250)
            # resize the image
            img = cv2.resize(img, new_size)
            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Apply a Gaussian blur to the image to reduce noise
            blur = cv2.GaussianBlur(gray, (5,5), 0)
            # Apply Canny edge detection to get the edges
            edges = cv2.Canny(blur, 100, 200)
            # Find the contours in the image
            contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            # Merge all the contours
            merged_contours = []
            for contour in contours:
                merged_contours += contour.tolist()
            oc = np.array([merged_contours])
            currvature_arr = get_curvatures(oc,radius_arr)

            curvature_arr = np.array(currvature_arr[0])

            x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
            y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales

            X, Y = np.meshgrid(x, y)

            hist_values = []
            for i in range(curvature_arr.shape[0]):
                counts, bin_edges = np.histogram(curvature_arr[i], bins=21) # adjust the number of bins as needed
                hist_values.append(counts)

            data = np.array([np.array(hist) for hist in hist_values])
            X2 = data.flatten().tolist()

            print(X2)

            features.append(X2)
            labels.append(foldername)


            end_time = time.time()
            total_time = end_time - start_time

        except:
            pass

        counter += 1
        
    print(f"Finished plant: {foldername}")
    
# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("lf_data.csv", index=False)

[3, 1, 3, 3, 9, 12, 53, 84, 112, 192, 372, 51, 18, 3, 4, 3, 2, 1, 0, 0, 3, 1, 0, 0, 5, 7, 20, 10, 25, 64, 65, 106, 182, 282, 76, 25, 38, 13, 2, 5, 2, 1, 1, 0, 0, 2, 1, 0, 0, 1, 7, 25, 73, 113, 261, 319, 108, 9, 2, 4, 0, 0, 3, 2, 0, 0, 0, 3, 2, 11, 45, 84, 233, 369, 130, 32, 5, 2, 4, 0, 0, 4, 2, 1, 3, 3, 2, 1, 28, 61, 109, 241, 306, 110, 25, 17, 15, 3, 0, 0, 1, 3, 0, 0, 1, 1, 1, 1, 12, 23, 37, 91, 145, 220, 225, 115, 31, 12, 3, 4, 4, 3, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 10, 12, 25, 68, 121, 153, 218, 206, 68, 35, 8, 1, 0, 0, 2, 2, 4, 6, 13, 22, 54, 81, 99, 165, 118, 128, 137, 50, 33, 11, 0, 2, 0, 2, 0, 2, 4, 1, 12, 13, 33, 51, 43, 65, 84, 110, 113, 121, 154, 49, 37, 19, 14, 1, 1, 0, 4, 2, 0, 0, 0, 0, 0, 2, 25, 58, 72, 96, 103, 137, 134, 140, 80, 49, 19, 9, 0, 3, 5, 19, 22, 24, 31, 55, 43, 55, 78, 71, 90, 116, 81, 86, 78, 32, 19, 16, 2, 2, 4, 2, 12, 44, 73, 61, 64, 79, 79, 68, 70, 115, 113, 60, 39, 22, 11, 6, 3, 1, 1, 6, 2, 0, 0, 0, 0, 8, 80, 122, 94, 141, 150, 130, 108, 57, 16, 6, 9, 3, 1, 

In [ ]:

start_time = time.time()
# Load the image
img = cv2.imread('/content/1249580491_0004.jpg')
# set the new size
new_size = (250, 250)
# resize the image
img = cv2.resize(img, new_size)
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Apply a Gaussian blur to the image to reduce noise
blur = cv2.GaussianBlur(gray, (5,5), 0)
# Apply Canny edge detection to get the edges
edges = cv2.Canny(blur, 100, 200)
# Find the contours in the image
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Merge all the contours
merged_contours = []
for contour in contours:
    merged_contours += contour.tolist()
oc = np.array([merged_contours])
currvature_arr = get_curvatures(oc,radius_arr)
end_time = time.time()
total_time = end_time - start_time
print(f"Total run time: {total_time} seconds")